In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np

In [11]:
empresas = pd.read_csv('empresas.csv')
infos = pd.read_csv('infos_empresas.csv')
cotacoes = pd.read_csv('cotacoes.csv')

empresas = empresas.drop(empresas.columns[0], axis=1)
infos = infos.drop(infos.columns[0], axis=1)
cotacoes = cotacoes.drop(cotacoes.columns[0], axis=1)